In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import json

In [2]:
# function courtesy of CryptoDataDownload.com
def fetch_daily_data(symbol):
    pair_split = symbol.split('/')  # symbol must be in format XXX/XXX ie. BTC/EUR
    symbol = pair_split[0] + '-' + pair_split[1]
    url = f'https://api.pro.coinbase.com/products/{symbol}/candles?granularity=86400'
    response = requests.get(url)
    if response.status_code == 200:  # check to make sure the response from server is good
        data = pd.DataFrame(json.loads(response.text), columns=['unix', 'low', 'high', 'open', 'close', 'volume'])
        data['date'] = pd.to_datetime(data['unix'], unit='s')  # convert to a readable date
        data['vol_fiat'] = data['volume'] * data['close']      # multiply the BTC volume by closing price to approximate fiat volume
                            
        # if we failed to get any data, print an error...otherwise write the file
        if data is None:
            print("Did not return any data from Coinbase for this symbol")
        else:
            data.to_csv(f'cb_{pair_split[0] + pair_split[1]}_daily.csv', index=False)
    else:
        print("Did not receieve OK response from Coinbase API")

<strong>Pulling Bitcoin</strong><br>
To get a baseline, first working with Bitcoin.

In [3]:
fetch_daily_data('BTC/USD')

In [4]:
btc = pd.read_csv('cb_BTCUSD_daily.csv', index_col='date', parse_dates=[0])

In [5]:
btc.head()

,unix,low,high,open,close,volume,vol_fiat
date,,,,,,,
2021-01-18,1610928000,34736.46,37402.00,35820.01,36215.85,14756.501118,5.344192e+08
2021-01-17,1610841600,33850.03,36860.00,36004.80,35820.00,19182.049347,6.871010e+08
2021-01-16,1610755200,35372.59,37948.00,36754.60,36006.94,20861.425452,7.511561e+08
2021-01-15,1610668800,34298.93,39697.00,39123.05,36754.67,36421.059188,1.338644e+09
2021-01-14,1610582400,36751.11,40127.66,37393.67,39125.14,31868.760494,1.246870e+09


In [6]:
btc.drop('unix', axis=1, inplace=True)

In [7]:
btc.info()

<class 'pandas.core.frame.DataFrame'>
Index: 300 entries, 2021-01-18 to 2020-03-25
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   low       300 non-null    float64
 1   high      300 non-null    float64
 2   open      300 non-null    float64
 3   close     300 non-null    float64
 4   volume    300 non-null    float64
 5   vol_fiat  300 non-null    float64
dtypes: float64(6)
memory usage: 16.4+ KB


In [8]:
btc.index = pd.to_datetime(btc.index)

In [9]:
btc['%change'] = (1 - (btc['high'] / btc['low'])) * -1

In [10]:
btc.head()

,low,high,open,close,volume,vol_fiat,%change
date,,,,,,,
2021-01-18,34736.46,37402.00,35820.01,36215.85,14756.501118,5.344192e+08,0.076736
2021-01-17,33850.03,36860.00,36004.80,35820.00,19182.049347,6.871010e+08,0.088921
2021-01-16,35372.59,37948.00,36754.60,36006.94,20861.425452,7.511561e+08,0.072808
2021-01-15,34298.93,39697.00,39123.05,36754.67,36421.059188,1.338644e+09,0.157383
2021-01-14,36751.11,40127.66,37393.67,39125.14,31868.760494,1.246870e+09,0.091876


In [11]:
btc = btc.sort_values('date')

# Expirimenting with Bitcoin

Before trying to create any models, the first test is to play with the data to see if it's theoretically possible to generate profits trading lows and highs. 

I will start by focusing on just the last month of data.

In [12]:
btc_test = btc[268:]

In [13]:
btc_test

,low,high,open,close,volume,vol_fiat,%change
date,,,,,,,
2020-12-18,22329.00,23280.00,22826.37,23137.76,24829.235818,5.744929e+08,0.042590
2020-12-19,22770.00,24200.00,23138.89,23849.99,20577.664412,4.907771e+08,0.062802
2020-12-20,23100.00,24300.00,23850.00,23476.51,14963.417871,3.512888e+08,0.051948
2020-12-21,21913.84,24118.75,23476.51,22729.40,25728.638560,5.847965e+08,0.100617
2020-12-22,22380.05,23839.78,22729.40,23823.27,18597.651230,4.430569e+08,0.065225
2020-12-23,22600.00,24090.00,23824.13,23228.35,23412.121720,5.438250e+08,0.065929
2020-12-24,22712.89,23785.00,23226.18,23717.96,16440.752951,3.899411e+08,0.047203
2020-12-25,23416.00,24770.95,23718.61,24704.71,15792.475834,3.901485e+08,0.057864
2020-12-26,24490.01,26822.00,24704.71,26475.35,24782.440156,6.561238e+08,0.095222


In [14]:
btc_test.drop(['open', 'close', 'volume', 'vol_fiat'], axis=1, inplace=True)

C:\Users\raide\OneDrive\Documents\python_projects\env\lib\site-packages\pandas\core\frame.py:4305: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [15]:
btc_test

,low,high,%change
date,,,
2020-12-18,22329.00,23280.00,0.042590
2020-12-19,22770.00,24200.00,0.062802
2020-12-20,23100.00,24300.00,0.051948
2020-12-21,21913.84,24118.75,0.100617
2020-12-22,22380.05,23839.78,0.065225
2020-12-23,22600.00,24090.00,0.065929
2020-12-24,22712.89,23785.00,0.047203
2020-12-25,23416.00,24770.95,0.057864
2020-12-26,24490.01,26822.00,0.095222


In [16]:
def buy_sell(data, amount=1000):
    '''Data: Takes a dataframe with date, high, low, and %change columns. 
    Amount: Takes a numerical value (default is 1000), which represents the money being invested.
    Returns a dataframe with appended columns indicating amount invested and amount returned (assuming daily buy/sell actions)'''
    
    data['investment'] = amount
    data['profit_loss'] = amount * data['%change']

In [17]:
buy_sell(btc_test)

<ipython-input-16-f43e4ecb326d>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['investment'] = amount
<ipython-input-16-f43e4ecb326d>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['profit_loss'] = amount * data['%change']


In [18]:
btc_test

,low,high,%change,investment,profit_loss
date,,,,,
2020-12-18,22329.00,23280.00,0.042590,1000,42.590353
2020-12-19,22770.00,24200.00,0.062802,1000,62.801932
2020-12-20,23100.00,24300.00,0.051948,1000,51.948052
2020-12-21,21913.84,24118.75,0.100617,1000,100.617236
2020-12-22,22380.05,23839.78,0.065225,1000,65.224609
2020-12-23,22600.00,24090.00,0.065929,1000,65.929204
2020-12-24,22712.89,23785.00,0.047203,1000,47.202712
2020-12-25,23416.00,24770.95,0.057864,1000,57.864281
2020-12-26,24490.01,26822.00,0.095222,1000,95.222093


In [19]:
total_profits = np.sum(btc_test['profit_loss'])

In [20]:
total_profits

3256.865029182056

Realistically, buying at the absolute low and selling at the absolute high is unrealistic. Still, if it were theoretically possible, with just reinvesting 1,000/day, one could make over 3,000 in one month profit (not including fees).<br>

Rather than erase this initial exploration, I will now re-create the function to include a varience.

In [21]:
def buy_sell_daily(data, amount=1000, variance=2):
    '''Data: Takes a dataframe with date, high, low, and %change columns. 
    Amount: Takes a numerical value (default is 1000), which represents the money being invested.
    Variance: Takes a whole number (recommended now more than 5), which represents the % difference to calculate. 
    Returns a dataframe with appended columns indicating amount invested and amount returned (assuming daily buy/sell actions)'''
    
    percent = variance / 100
    data['investment'] = amount
    data['profit_loss'] = amount * (data['%change'] - percent)

In [22]:
buy_sell_daily(btc_test, 1000, 5)

<ipython-input-21-48d3e43b399a>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['investment'] = amount
<ipython-input-21-48d3e43b399a>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['profit_loss'] = amount * (data['%change'] - percent)


In [23]:
btc_test

,low,high,%change,investment,profit_loss
date,,,,,
2020-12-18,22329.00,23280.00,0.042590,1000,-7.409647
2020-12-19,22770.00,24200.00,0.062802,1000,12.801932
2020-12-20,23100.00,24300.00,0.051948,1000,1.948052
2020-12-21,21913.84,24118.75,0.100617,1000,50.617236
2020-12-22,22380.05,23839.78,0.065225,1000,15.224609
2020-12-23,22600.00,24090.00,0.065929,1000,15.929204
2020-12-24,22712.89,23785.00,0.047203,1000,-2.797288
2020-12-25,23416.00,24770.95,0.057864,1000,7.864281
2020-12-26,24490.01,26822.00,0.095222,1000,45.222093


In [24]:
total_profits = np.sum(btc_test['profit_loss'])
total_profits

1656.865029182056

Even assuming a liberal 5% lower return than the difference between high and low prices, we still get over 1,500 in this month period, which is more than we would get from buying, holding, and finally selling after 30 days. 

With that in mind, the next test is to see how it performs in other month-long periods. 

If we can reproduce this again, we'll move into a new notebook to start working on features and building a machine learning model that can predict the highs/lows. 

## Testing Other Coins
Here we can test additional coins in the future.